In [96]:
import pandas as pd
import numpy as np

In [97]:
data = pd.read_excel("URINELOTS.xlsx")
#data.head()

In [98]:
data.shape

(146, 16)

#### DATA cleaning

In [99]:
data = data.dropna(subset=['RAMJA Results','AIIMS Result culture'])

In [100]:
import re
def cleanup(text):
    text = text.upper()
    text = text.replace(".", " ")
    text = text.replace(" AND ", " ")
    text = re.sub(r'\s+', ' ', text)  # Use re.sub for regular expression replacement
    text = text.strip()
    return text


In [101]:
bacteriatypes = {
    "COLI" : 'Escherichia coli', 
    "PNEUMO" : 'Klebsiella pneumoniae',
    "AERUGI" : 'Pseudomonas aeruginosa',
    "BAUMAN" : 'Acinetobacter baumannii', 
    "MIRABIL" : 'Proteus mirabilis', 
    "FAECALI" : 'Enterococcus faecalis', 
    "AUREUS" : 'Staphylococcus aureus', 
    "SAPROPHY" : 'Staphylococcus saprophyticus',
    "STERI" : 'Sterile'
}
#['Escherichia coli','Klebsiella pneumoniae','Pseudomonas aeruginosa','Acinetobacter baumannii','Proteus mirabilis','Enterococcus faecalis','Staphylococcus aureus','Staphylococcus saprophyticus','Sterile']

#### RAMJA results

In [102]:
resultsRAMJA = data[['UHID','RAMJA Results']]

In [103]:
resultsRAMJA['cleanRAMJA'] = resultsRAMJA['RAMJA Results'].apply(cleanup)

C:\Users\neelesh\AppData\Local\Temp\ipykernel_9308\766998183.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultsRAMJA['cleanRAMJA'] = resultsRAMJA['RAMJA Results'].apply(cleanup)


In [104]:
resultsRAMJA.head()

,UHID,RAMJA Results,cleanRAMJA
0,20220065432,Enterococcus faecalis,ENTEROCOCCUS FAECALIS
2,20230125389,"E.Coli, K.pneumoniae, P. aeruginosa, A. bauman...","E COLI, K PNEUMONIAE, P AERUGINOSA, A BAUMANNI..."
3,20180022685,"K.pneumoniae, E. faecalis, P. mirabilis, A. b...","K PNEUMONIAE, E FAECALIS, P MIRABILIS, A BAUMA..."
5,20240012395,"Polymicrobial (E.coli, A. baumannii, P.mirabil...","POLYMICROBIAL (E COLI, A BAUMANNII, P MIRABILI..."
6,20220166372,"Pseudomonas, P.mirabilis, E.Faecalis, S. aureus","PSEUDOMONAS, P MIRABILIS, E FAECALIS, S AUREUS"


In [105]:
def check(mainstring, s1, s2):
    if (mainstring.count(s1) > 0 or mainstring.count(s2) > 0):
        #print("YES")
        return(1)
    else:
        #print("NO")
        return(0)


mainstring = "E COLI, K PNEUMONIAE, P AERUGINOSA, A BAUMANNI"
s1 = "E COLI"
s2 = "K PNEUMONIAE"
check(mainstring, s2, s1)

1

In [106]:
genus = ["ESCHER","KLEBS","PSEUDO","ACINE","PROT","ENTER","AURE","SAPRO","STERI"]
species = ["COLI", "PNEUMO", "AERUGI", "BAUMAN", "MIRABIL", "FAECALI", "AUREUS", "SAPROPHY", "STERI"]
for s1, s2 in zip(genus,species):
    resultsRAMJA[s2] = resultsRAMJA['cleanRAMJA'].apply(lambda x: check(x, s1, s2))

C:\Users\neelesh\AppData\Local\Temp\ipykernel_9308\1057333925.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultsRAMJA[s2] = resultsRAMJA['cleanRAMJA'].apply(lambda x: check(x, s1, s2))
C:\Users\neelesh\AppData\Local\Temp\ipykernel_9308\1057333925.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultsRAMJA[s2] = resultsRAMJA['cleanRAMJA'].apply(lambda x: check(x, s1, s2))
C:\Users\neelesh\AppData\Local\Temp\ipykernel_9308\1057333925.py:4: SettingWithCopyWarning: 
A value is trying to be set on 

In [71]:
#resultsRAMJA.head()

In [107]:
resultsRAMJA.rename(columns=bacteriatypes, inplace=True)

C:\Users\neelesh\AppData\Local\Temp\ipykernel_9308\78547064.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultsRAMJA.rename(columns=bacteriatypes, inplace=True)


In [74]:
#resultsRAMJA.head()

In [108]:
columns_to_modify = ['Escherichia coli','Klebsiella pneumoniae','Pseudomonas aeruginosa','Acinetobacter baumannii','Proteus mirabilis','Enterococcus faecalis','Staphylococcus aureus','Staphylococcus saprophyticus','Sterile']

In [109]:
resultsRAMJA.rename(columns={col: f"{col}_R" for col in columns_to_modify}, inplace=True)

C:\Users\neelesh\AppData\Local\Temp\ipykernel_9308\1583815861.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultsRAMJA.rename(columns={col: f"{col}_R" for col in columns_to_modify}, inplace=True)


In [110]:
resultsRAMJA.head()

,UHID,RAMJA Results,cleanRAMJA,Escherichia coli_R,Klebsiella pneumoniae_R,Pseudomonas aeruginosa_R,Acinetobacter baumannii_R,Proteus mirabilis_R,Enterococcus faecalis_R,Staphylococcus aureus_R,Staphylococcus saprophyticus_R,Sterile_R
0,20220065432,Enterococcus faecalis,ENTEROCOCCUS FAECALIS,0,0,0,0,0,1,0,0,0
2,20230125389,"E.Coli, K.pneumoniae, P. aeruginosa, A. bauman...","E COLI, K PNEUMONIAE, P AERUGINOSA, A BAUMANNI...",1,1,1,1,0,0,1,0,0
3,20180022685,"K.pneumoniae, E. faecalis, P. mirabilis, A. b...","K PNEUMONIAE, E FAECALIS, P MIRABILIS, A BAUMA...",0,1,0,1,1,1,1,0,0
5,20240012395,"Polymicrobial (E.coli, A. baumannii, P.mirabil...","POLYMICROBIAL (E COLI, A BAUMANNII, P MIRABILI...",1,0,0,1,1,1,1,0,0
6,20220166372,"Pseudomonas, P.mirabilis, E.Faecalis, S. aureus","PSEUDOMONAS, P MIRABILIS, E FAECALIS, S AUREUS",0,0,1,0,1,1,1,0,0


#### AIIMS results

In [111]:
resultsAIIMS = data[['UHID','AIIMS Result culture']]

In [112]:
resultsAIIMS['cleanAIIMS'] = resultsAIIMS['AIIMS Result culture'].apply(cleanup)

C:\Users\neelesh\AppData\Local\Temp\ipykernel_9308\265187514.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultsAIIMS['cleanAIIMS'] = resultsAIIMS['AIIMS Result culture'].apply(cleanup)


In [80]:
#resultsAIIMS.head()

In [113]:
genus = ["ESCHER","KLEBS","PSEUDO","ACINE","PROT","ENTER","AURE","SAPRO","STERI"]
species = ["COLI", "PNEUMO", "AERUGI", "BAUMAN", "MIRABIL", "FAECALI", "AUREUS", "SAPROPHY", "STERI"]
for s1, s2 in zip(genus,species):
    resultsAIIMS[s2] = resultsAIIMS['cleanAIIMS'].apply(lambda x: check(x, s1, s2))

C:\Users\neelesh\AppData\Local\Temp\ipykernel_9308\2996149377.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultsAIIMS[s2] = resultsAIIMS['cleanAIIMS'].apply(lambda x: check(x, s1, s2))
C:\Users\neelesh\AppData\Local\Temp\ipykernel_9308\2996149377.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultsAIIMS[s2] = resultsAIIMS['cleanAIIMS'].apply(lambda x: check(x, s1, s2))
C:\Users\neelesh\AppData\Local\Temp\ipykernel_9308\2996149377.py:4: SettingWithCopyWarning: 
A value is trying to be set on 

In [85]:
#resultsAIIMS.head()

In [114]:
resultsAIIMS.rename(columns=bacteriatypes, inplace=True)

C:\Users\neelesh\AppData\Local\Temp\ipykernel_9308\4113831117.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultsAIIMS.rename(columns=bacteriatypes, inplace=True)


In [88]:
#resultsAIIMS.head()

In [115]:
resultsAIIMS.rename(columns={col: f"{col}_A" for col in columns_to_modify}, inplace=True)

C:\Users\neelesh\AppData\Local\Temp\ipykernel_9308\3656104579.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultsAIIMS.rename(columns={col: f"{col}_A" for col in columns_to_modify}, inplace=True)


In [116]:
resultsAIIMS.head()

,UHID,AIIMS Result culture,cleanAIIMS,Escherichia coli_A,Klebsiella pneumoniae_A,Pseudomonas aeruginosa_A,Acinetobacter baumannii_A,Proteus mirabilis_A,Enterococcus faecalis_A,Staphylococcus aureus_A,Staphylococcus saprophyticus_A,Sterile_A
0,20220065432,Enterococcus faecalis,ENTEROCOCCUS FAECALIS,0,0,0,0,0,1,0,0,0
2,20230125389,E.coli,E COLI,1,0,0,0,0,0,0,0,0
3,20180022685,CONTAMINATION,CONTAMINATION,0,0,0,0,0,0,0,0,0
5,20240012395,E.coli,E COLI,1,0,0,0,0,0,0,0,0
6,20220166372,E.coli & Pseudomnas,E COLI & PSEUDOMNAS,1,0,1,0,0,0,0,0,0


#### Combining both AIIMS and RAMJA results

In [117]:
results = pd.merge(resultsRAMJA, resultsAIIMS, on="UHID")

In [118]:
results.head()

,UHID,RAMJA Results,cleanRAMJA,Escherichia coli_R,Klebsiella pneumoniae_R,Pseudomonas aeruginosa_R,Acinetobacter baumannii_R,Proteus mirabilis_R,Enterococcus faecalis_R,Staphylococcus aureus_R,...,cleanAIIMS,Escherichia coli_A,Klebsiella pneumoniae_A,Pseudomonas aeruginosa_A,Acinetobacter baumannii_A,Proteus mirabilis_A,Enterococcus faecalis_A,Staphylococcus aureus_A,Staphylococcus saprophyticus_A,Sterile_A
0,20220065432,Enterococcus faecalis,ENTEROCOCCUS FAECALIS,0,0,0,0,0,1,0,...,ENTEROCOCCUS FAECALIS,0,0,0,0,0,1,0,0,0
1,20230125389,"E.Coli, K.pneumoniae, P. aeruginosa, A. bauman...","E COLI, K PNEUMONIAE, P AERUGINOSA, A BAUMANNI...",1,1,1,1,0,0,1,...,E COLI,1,0,0,0,0,0,0,0,0
2,20180022685,"K.pneumoniae, E. faecalis, P. mirabilis, A. b...","K PNEUMONIAE, E FAECALIS, P MIRABILIS, A BAUMA...",0,1,0,1,1,1,1,...,CONTAMINATION,0,0,0,0,0,0,0,0,0
3,20240012395,"Polymicrobial (E.coli, A. baumannii, P.mirabil...","POLYMICROBIAL (E COLI, A BAUMANNII, P MIRABILI...",1,0,0,1,1,1,1,...,E COLI,1,0,0,0,0,0,0,0,0
4,20220166372,"Pseudomonas, P.mirabilis, E.Faecalis, S. aureus","PSEUDOMONAS, P MIRABILIS, E FAECALIS, S AUREUS",0,0,1,0,1,1,1,...,E COLI & PSEUDOMNAS,1,0,1,0,0,0,0,0,0


In [119]:
#results.to_excel("URINELOT1.xlsx")

#### Compare few results

In [121]:
print("RAMJA : \n", resultsRAMJA["Escherichia coli_R"].value_counts())
print("AIIMS : \n",resultsAIIMS["Escherichia coli_A"].value_counts())

RAMJA : 
 0    75
1    43
Name: Escherichia coli_R, dtype: int64
AIIMS : 
 0    100
1     18
Name: Escherichia coli_A, dtype: int64
